In [2]:
# code to clean up calls.csv and move it to a SQL table

import numpy as np
import pandas as pd

calls = pd.read_csv("data/Call_Data.csv")

def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split(" ")
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    df["time"] = yearmonthday.apply(lambda x: x[3])
    return df

calls = callstime(calls,"Arrived Time")
calls17 = calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])]

# calculate and add column for epoch time
bar = pd.to_datetime( calls17["Arrived Time"], format="%b %d %Y %I:%M:%S:%f%p" )
epoch_second2 = bar.map(lambda x: x.value/1e9)
calls17["epoch_calls"] = epoch_second2

rs = calls17["CAD Event Number"].value_counts()
two = []
for v, i in zip(rs.values, rs.index):
    if v >1:
        two.append(i)
# len(two) == 362  
st = [x > 1 for x in rs]
dt =rs[st]

t = calls17[calls17["CAD Event Number"].isin(two)]
indt = t[t["Event Clearance Description"] == "-"].index
indt

calls17.drop(indt, axis=0, inplace=True)

#remove repetitive columns
calls_tosql = calls17.drop(columns =["year","day", "month", "time", "Precinct", "Sector", "Arrived Time"])
calls_tosql.set_index("epoch_calls", inplace=True)

# move calls to sqlite table - DONT REPEAT
#calls_tosql.to_sql("log", con=cnx, index=True, if_exists="replace")


/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3986668 entries, 0 to 3986667
Data columns (total 15 columns):
CAD Event Number               int64
Event Clearance Description    object
Call Type                      object
Priority                       int64
Initial Call Type              object
Final Call Type                object
Original Time Queued           object
Arrived Time                   object
Precinct                       object
Sector                         object
Beat                           object
year                           object
day                            object
month                          int64
time                           object
dtypes: int64(3), object(12)
memory usage: 456.2+ MB


In [8]:
calls.groupby("Beat")["Initial Call Type"].value_counts()

Beat  Initial Call Type                                 
1     -ASSIGNED DUTY - DETAIL BY SUPERVISOR                    1
99    TRAFFIC STOP - OFFICER INITIATED ONVIEW               1431
      -OUT AT RANGE                                          804
      ASSIST OTHER AGENCY - ROUTINE SERVICE                  658
      FOLLOW UP                                              615
      WARRANT - FELONY PICKUP                                563
      WARRANT - MISD WARRANT PICKUP                          352
      -ASSIGNED DUTY - IN-SERVICE TRAINING                   313
      AUTO RECOVERY                                          304
      SUSPICIOUS PERSON, VEHICLE OR INCIDENT                 280
      PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY           247
      SUSPICIOUS STOP - OFFICER INITIATED ONVIEW             236
      WARRANT PICKUP - FROM OTHER AGENCY                     199
      OFF DUTY EMPLOYMENT                                    175
      -ASSIGNED DUTY - DETAIL BY 